In [ ]:
!pip install langchain-community Chroma Chromadb pymuPDF pyPDF2 sentence-transformers langchain pypdf streamlit

In [2]:
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
import os

In [6]:
# Directorio que contiene los archivos PDF
pdf_directory = 'doc'

# Lista para almacenar documentos
all_docs = []

# Iterar sobre todos los archivos en el directorio
for filename in os.listdir(pdf_directory):
    if filename.lower().endswith('.pdf'):
        file_path = os.path.join(pdf_directory, filename)
        print(f"Cargando archivo: {file_path}")

        # Cargar el PDF
        loader = PyPDFLoader(file_path)
        docs = loader.load()

        # Agregar los documentos cargados a la lista
        all_docs.extend(docs)

# Imprimir la cantidad total de partes del PDF
print(f"Cantidad total de partes del PDF: {len(all_docs)}")

Cargando archivo: doc\preguntas.pdf
Cargando archivo: doc\respuestas.pdf
Cargando archivo: doc\teoria.pdf
Cantidad total de partes del PDF: 53


In [7]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #modelo de embeddings gratis

c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, a

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) #usamos menos tokens que en el anterior debido a que este modelo es inferior
splits = text_splitter.split_documents(all_docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory='./vectordb')

retriever = vectorstore.as_retriever()

In [9]:
retriever.get_relevant_documents('¿Cuál es la idea básica del proyecto?')

c:\Users\Pablo\Desktop\RAG\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'page': 0, 'source': '/content/pdf_files/respuestas.pdf'}, page_content='1.\nPresentación\ndel\nProyecto\n1.1.\nDescripción\ndel\nNegocio\nAyuda\npara\nResponder:\n●\nIdea\nBásica\ndel\nProyecto:\no\nPiensa\nen\nqué\noportunidad\nde\nmercado\nestás\naprovechando\ncon\ntu\nidea\nde \nnegocio.\nSegún\nel\ndocumento,\nla\nidea\ndebe\ncubrir\nuna\ndemanda\nno \nsatisfecha\no\nmejorar\nun\nproducto/servicio\nexistente.\no\nDescribe\nel\nproducto\no\nservicio\ndestacando\nsi\nes\nalgo\nnuevo\nen\nel \nmercado\no\nuna\ninnovación\nsobre\nalgo\nya\nexistente.\n●\nProductos\no\nServicios:\no\nSi\nes\nun\nproducto,\ndescribe\nsus\ncaracterísticas,'),
 Document(metadata={'page': 0, 'source': 'doc\\respuestas.pdf'}, page_content='1.\nPresentación\ndel\nProyecto\n1.1.\nDescripción\ndel\nNegocio\nAyuda\npara\nResponder:\n●\nIdea\nBásica\ndel\nProyecto:\no\nPiensa\nen\nqué\noportunidad\nde\nmercado\nestás\naprovechando\ncon\ntu\nidea\nde \nnegocio.\nSegún\nel\ndocumento,\nla\nidea

In [19]:
import shutil

# Ruta de la carpeta que deseas comprimir
folder_path = '/content/vectordb'  # Cambia esto a la ruta de tu carpeta
zip_path = '/content/vectordb.zip'

# Comprimir la carpeta
shutil.make_archive('/content/vectordb', 'zip', folder_path)

'/content/vectordb.zip'